In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = "{:.4f}".format
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,10)
plt.style.use("~/.dracula.mplstyle")
import statsmodels.api as sm

In [2]:
class DataImport:
    
    def __init__(self, filename, add_ind=[], na_method="interpolate"):
        self.xlsx = pd.ExcelFile(filename)
        self.add_ind = add_ind # Additional indices to try to 
        
        print(f"Read in <{filename}>.")
        self.sheets = []
        print('-'*20, "Reading Sheets ", '-'*20)
        self.read_sheets()
        print('-'*20, "Setting Indices", '-'*20)
        self.set_indices()
        print('-'*20, "Checking Nulls ", '-'*20)
        self.check_nulls(na_method)
        print("\n\n")
    
    def read_sheets(self):
        for i, s in enumerate(self.xlsx.sheet_names):
            read_sheet = self.xlsx.parse(s)
            print(f"Sheet {i}: {s}; shape={read_sheet.shape}")
            self.sheets.append(read_sheet)
    
    def set_indices(self):
        for s in self.sheets:
            for d in ["date", "Date", "DATE", "Unnamed: 0"]+self.add_ind:
                try:
                    s.set_index(d, inplace=True) # Date as index
                    break
                except KeyError:
                    continue
    
    def check_nulls(self, method):
        n = 0
        for i, s in enumerate(self.sheets[1:]):
            has_nulls = True
            while has_nulls:
                if sum(s.isna().sum())==0: # Check for nulls
                    has_nulls = False
                else:
                    if method=="interpolate":
                        print(f"Null values found in <{i+1}>. Interpolating linearly.")
                        s = s.interpolate() # Linerly interpolate nulls
                        self.sheets[i+1] = s
                    elif method=="drop":
                        print(f"Null values found in <{i+1}>. Dropping.")
                        s = s.dropna()
                        self.sheets[i+1] = s
                    elif method=="none":
                        break
                    n += 1
                if n>i+1:
                    print("Something broke...")
                    n=i+1
                    break
        if n==0: print("No nulls found :)")

ltcm = DataImport("ltcm.xlsx", na_method="drop")
# display(ltcm.sheets[0])
ex2 = ltcm.sheets[1]
mapper = {
    "Exhibit 2     Fund Capital and Monthly Returns (June, 1994 to July, 1998)": "date",
    "Unnamed: 1": "fund_cap",
    "Unnamed: 2": "gross",
    "Unnamed: 3": "net",
    "Unnamed: 4": "cumulative",
}
ex2.rename(columns=mapper, inplace=True)
ex2.set_index("date", inplace=True)
for c in ex2.columns:
    ex2[c] = pd.to_numeric(ex2[c])

gmo = DataImport("gmo.xlsx", na_method="none")
# display(gmo.sheets[0])
gmo.sheets[2].index += pd.DateOffset(1)
gmo.sheets[3].index += pd.DateOffset(1)
gmo.sheets[2].index += pd.DateOffset(months=-1)
gmo.sheets[3].index += pd.DateOffset(months=-1)
ex2 = ex2.join(gmo.sheets[2]["SPY"])
ex2 = ex2.join(gmo.sheets[3]["US3M"])
ex2[["gross", "net", "SPY"]] = ex2[["gross", "net", "SPY"]].subtract(ex2["US3M"], axis=0)

display(ex2.head())

Read in <ltcm.xlsx>.
-------------------- Reading Sheets  --------------------
Sheet 0: Copyright; shape=(4, 1)
Sheet 1: Exhibit 2; shape=(60, 5)
Sheet 2: Exhibit 4; shape=(54, 5)
-------------------- Setting Indices --------------------
-------------------- Checking Nulls  --------------------
Null values found in <1>. Dropping.
Null values found in <2>. Dropping.



Read in <gmo.xlsx>.
-------------------- Reading Sheets  --------------------
Sheet 0: descriptions; shape=(6, 4)
Sheet 1: signals; shape=(345, 4)
Sheet 2: returns (total); shape=(345, 3)
Sheet 3: risk-free rate; shape=(345, 2)
-------------------- Setting Indices --------------------
-------------------- Checking Nulls  --------------------
No nulls found :)





,fund_cap,gross,net,cumulative,SPY,US3M
date,,,,,,
1994-03-01,1.1000,-0.0140,-0.0160,0.9900,-0.0449,0.0030
1994-04-01,1.1000,0.0107,0.0047,1.0000,0.0079,0.0033
1994-05-01,1.2000,0.0644,0.0494,1.0500,0.0123,0.0036
1994-06-01,1.2000,-0.0425,-0.0326,1.0200,-0.0264,0.0036
1994-07-01,1.4000,0.1123,0.0803,1.1000,0.0287,0.0037


# 2. LTCM Risk Decomposition

On Canvas, find the data file, “ltcm exhibits data.xlsx”. Get the gross and net (total) returns of LTCM from “Exhibit 2”.

Get the returns on SPY as well as the risk-free rate from the file, “gmo analysis data”.

#### 1. Summary stats.

(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.)

In [3]:
def pivot_summaries(data, year_mask=None, pi=0.05, nper=12):
    if not(year_mask):
        year_mask = [(str(min(data.index).year), str(max(data.index).year))]
    
    data_melt = pd.melt(data, ignore_index=False).sort_index()
    
    pivlist = []
    for start, end in year_mask:
        def me(x): return nper * np.mean(x)
        def sd(x): return np.sqrt(nper) * np.std(x, ddof=1) # account for degrees of freedom
        def sr(x): return np.sqrt(nper) * np.mean(x) / np.std(x, ddof=1)
        af = [me, sd, sr]
        af_names = ["Mean", "Vol", "Sharpe"]
        
        piv = pd.pivot_table(data_melt[start:end], index="variable", values="value",
                             aggfunc=af)
        piv.columns = pd.MultiIndex.from_arrays([[f"{start}-{end}"] * len(af), af_names],
                                                names=["year", "measure"])
        pivlist.append(piv)

    summ = pd.concat(pivlist, axis=1)
    return summ

ps = pivot_summaries(ex2[["gross", "net", "SPY"]])
display(ps)

year     1994-1998              
measure       Mean    Vol Sharpe
variable                        
SPY         0.1738 0.1123 1.5479
gross       0.2421 0.1362 1.7769
net         0.1554 0.1118 1.3901

(b) Report the skewness, kurtosis, and (historic) VaR(.05).

In [4]:
def tailrisk(v, prob, logret=False, exkurt=True):
    """Arguments:
    v: pd.DataFrame containing columns of returns
    prob: float percentile for VaR
    excess: bool whether kurtosis of normal = 0 (True) or = 3 (False)
    """
    try:
        c = v.columns
    except AttributeError:
        c = ["Portfolio"]
    
    mi = v.min()
    sk = v.skew()
    ku = v.kurtosis() - 3 * exkurt
    var = v.quantile(prob)
    
    cvar = []
    mask =  v < var
    for name, col in mask.iteritems():
        below = v[name].loc[col]
        cvar.append(sum(below)/len(below))
    
    cvar = pd.Series(cvar, index=c)
    
    if logret:
        cumu = np.exp(v.cumsum())
    else:
        cumu = (v+1).cumprod()
    from_peak = (cumu - cumu.cummax()) / cumu.cummax()

    pl, rl, dl = [], [], []
    trough = from_peak.idxmin()
    for col, date in trough.iteritems():
        peak = max(v.loc[(from_peak.index < date) & (from_peak[col] == 0), col].index)
        try:
            reco = min(v.loc[(from_peak.index > date) & (from_peak[col] == 0), col].index)
        except ValueError:
            reco = None
        
        if logret:
            # This may not work right at the moment...
            draw = np.log(cumu.loc[date,col] / cumu.loc[peak,col])
        else:
            draw = (cumu.loc[date,col] - cumu.loc[peak,col]) / cumu.loc[peak,col]

        pl.append(peak); rl.append(reco); dl.append(draw)

    dl = pd.Series(dl, index=c)
    pl = pd.Series(pl, index=c)
    rl = pd.Series(rl, index=c)
    
    ret = pd.DataFrame([mi, sk, ku, var, cvar, dl],
                       index=["min", "skew", "kurtosis", f"var_{prob}", f"cvar_{prob}", "drawdown"],
                       columns=c)
    dra = pd.DataFrame([pl, trough, rl],
                       index=["peak", "trough", "recovery"],
                       columns=c)
    ret = pd.concat([ret], axis=0).T
    
    return ret

tr = tailrisk(ex2[["gross", "net", "SPY"]], 0.05, logret=False, exkurt=False)

display(tr)

,min,skew,kurtosis,var_0.05,cvar_0.05,drawdown
gross,-0.1053,-0.2877,1.5866,-0.0304,-0.0730,-0.1689
net,-0.1053,-0.8102,2.9269,-0.0264,-0.0687,-0.1761
SPY,-0.0562,-0.4335,-0.3620,-0.0464,-0.0514,-0.0562


(c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

pass

#### 2. Using the series of net LTCM excess returns, denoted r ̃, estimate the following regression:

(a) Report α and βm. Report the R2 stat.

(b) From this regression, does LTCM appear to be a “closet indexer”?

pass

(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

yes alpha pass

In [5]:
res = sm.OLS(ex2["net"], sm.add_constant(ex2["SPY"])).fit()
display(res.summary())

/Users/plee/miniconda3/envs/finmath/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    net   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9866
Date:                Sun, 05 Dec 2021   Prob (F-statistic):              0.325
Time:                        23:52:46   Log-Likelihood:                 107.80
No. Observations:                  53   AIC:                            -211.6
Df Residuals:                      51   BIC:                            -207.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0110      0.005      2.254      0.029       0.001       0.021
SPY            0.1371      0.138      0.993      0.325      -0.140       0.414
==============================================================================
Omnibus:                       14.910   Durbin-Watson:                   1.810
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               25.291
Skew:                          -0.838   Prob(JB):                     3.22e-06
Kurtosis:                       5.940   Cond. No.                         31.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 3. Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns:

(a) Report β1, β2, and the R2 stat.

In [6]:
exo = pd.concat([pd.Series(np.ones(len(ex2)), name="const", index=ex2.index), ex2["SPY"],
                 (ex2["SPY"]**2).rename("SPY_SQ")], axis=1)

res = sm.OLS(ex2["net"], exo).fit()
display(res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    net   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.6232
Date:                Sun, 05 Dec 2021   Prob (F-statistic):              0.540
Time:                        23:52:46   Log-Likelihood:                 107.95
No. Observations:                  53   AIC:                            -209.9
Df Residuals:                      50   BIC:                            -204.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0129      0.006      2.096      0.041       0.001       0.025
SPY            0.1669      0.150      1.111      0.272      -0.135       0.469
SPY_SQ        -1.9267      3.682     -0.523      0.603      -9.323       5.469
==============================================================================
Omnibus:                       15.616   Durbin-Watson:                   1.866
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.759
Skew:                          -0.880   Prob(JB):                     1.55e-06
Kurtosis:                       6.004   Cond. No.                         825.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

(b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

pass

(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

short market put: positive delta, negative gamma

(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

pass

#### 4. Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:

(a) Report β, βu, βd, and the R2 stat.

In [7]:
k1 = 0.03
k2 = -k1

exo = pd.concat([pd.Series(np.ones(len(ex2)), name="const", index=ex2.index), ex2["SPY"],
                 ex2["SPY"].apply(lambda x: max(x-k1, 0)).rename("SPY_UP"),
                 ex2["SPY"].apply(lambda x: max(k2-x, 0)).rename("SPY_DOWN")], axis=1)

res = sm.OLS(ex2["net"], exo).fit()
display(res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    net   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9595
Date:                Sun, 05 Dec 2021   Prob (F-statistic):              0.419
Time:                        23:52:46   Log-Likelihood:                 108.81
No. Observations:                  53   AIC:                            -209.6
Df Residuals:                      49   BIC:                            -201.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0084      0.006      1.417      0.163      -0.004       0.020
SPY            0.4666      0.277      1.687      0.098      -0.089       1.023
SPY_UP        -0.7821      0.629     -1.244      0.219      -2.045       0.481
SPY_DOWN       1.2896      1.157      1.115      0.270      -1.036       3.615
==============================================================================
Omnibus:                       18.184   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.179
Skew:                          -0.980   Prob(JB):                     2.30e-08
Kurtosis:                       6.477   Cond. No.                         276.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

(b) Is LTCM long or short the call-like factor? And the put-like factor?

pass

(c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

pass

(d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?

pass

In [8]:
fxxl = DataImport("fx.xlsx", na_method="drop")
display(fxxl.sheets[0])
rates = fxxl.sheets[1] / 12
fx = fxxl.sheets[2]
rates = np.log(rates+1)
fx = np.log(fx)
display(rates, fx)

Read in <fx.xlsx>.
-------------------- Reading Sheets  --------------------
Sheet 0: descriptions; shape=(9, 2)
Sheet 1: risk-free rates; shape=(274, 6)
Sheet 2: fx rates; shape=(274, 5)
-------------------- Setting Indices --------------------
-------------------- Checking Nulls  --------------------
No nulls found :)





,Ticker Description
Unnamed: 0,
USD3M,"3-Month LIBOR, USD"
GBP3M,"3-Month LIBOR, GBP"
EUR3M,"3-Month LIBOR, EUR"
CHF3M,"3-Month LIBOR, CHF"
JPY3M,"3-Month LIBOR, JPY"
USUK,"Spot FX, US per UK (GBP)"
USEU,"Spot FX, US per EU (EUR)"
USSZ,"Spot FX, US per SZ (CHF)"
USJPY,"Spot FX, US per JP (JPY)"


,USD3M,GBP3M,EUR3M,CHF3M,JPY3M
DATE,,,,,
1999-01-31,0.0041,0.0048,0.0026,0.0011,0.0004
1999-02-28,0.0042,0.0045,0.0026,0.0011,0.0002
1999-03-31,0.0042,0.0044,0.0025,0.0010,0.0002
1999-04-30,0.0041,0.0044,0.0022,0.0008,0.0001
1999-05-31,0.0042,0.0045,0.0021,0.0009,0.0001
...,...,...,...,...,...
2021-06-30,0.0001,0.0001,-0.0005,-0.0006,-0.0001
2021-07-31,0.0001,0.0001,-0.0005,-0.0006,-0.0001
2021-08-31,0.0001,0.0001,-0.0005,-0.0006,-0.0001


,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948
...,...,...,...,...
2021-06-30,0.3225,0.1696,0.0776,-4.7100
2021-07-31,0.3302,0.1709,0.0986,-4.6977
2021-08-31,0.3182,0.1655,0.0876,-4.7009


In [9]:
dspot = fx - fx.shift(1)
fore = dspot.add(rates[rates.columns[1:]].values).dropna()
forex = fore.subtract(rates["USD3M"], axis=0).dropna()
display(forex)

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0261,-0.0352,-0.0259,-0.0270
1999-03-31,0.0073,-0.0188,-0.0223,-0.0017
1999-04-30,-0.0031,-0.0248,-0.0354,-0.0124
1999-05-31,-0.0038,-0.0156,-0.0049,-0.0162
1999-06-30,-0.0162,-0.0130,-0.0213,-0.0049
...,...,...,...,...
2021-06-30,-0.0273,-0.0294,-0.0290,-0.0112
2021-07-31,0.0077,0.0008,0.0202,0.0120
2021-08-31,-0.0120,-0.0060,-0.0117,-0.0034


# 3. The FX Carry Trade

Find an Excel data file, “fx carry data.xlsx”. The file has two sets of data:

• Risk-free rates across 5 currencies, as measured by annualized 3-month LIBOR rates.

• Spot FX rates, as direct quotes to the USD. (Note that all currencies are quoted as USD per the foreign currency.)

For use in the homework, note the following:

• For risk-free rate data, rf,i , the rate is known and reported in the data at time t. Namely,
any given date t in the data file is reporting both Si and rf,i .

• The theory says to use log risk-free rates. You have the risk-free rate in levels: use the following
equation to convert them:

rf,i = ln(1 + rf,i ) t,t+1 t,t+1

• The theory says to use log spot FX prices. You have the FX prices in levels, so directly take their logarithims:
sit = ln(Sti)

#### 1. The Static Carry Trade

Define the log return of holding the foreign currency using log values of the risk-free rate and log values of the FX rates:

ri ≡ si − si + rf,i t+1 t+1 t t,t+1

Then the excess log return relative to USD, is expressed as

For each foreign currency, i, calculate the excess log return series,  ̃rt+1. Report the following
stats, (based on the excess log returns.) Annualize them.

(a) mean

(b) volatility

(c) Sharpe ratio

What differences do you see across currencies?

All currencies have low mean, indicating relative stability with respect to one another. The European currencies move on a similar magnitude (~0.0035-0.0045) due to the exchange rate with dollars being relatively close to 1. EUR, JPY, and GBP have all weakened agains USD as indicated by the negative mean whereas CHF has strengthened. All currencies have annualized volatility of around 9%. JPY has weakened the most substantially compared with the other currencies.

In [10]:
ps = pivot_summaries(forex)
display(ps)

year     1999-2021               
measure       Mean    Vol  Sharpe
variable                         
USEU       -0.0045 0.0948 -0.0474
USJP       -0.0180 0.0916 -0.1970
USSZ        0.0040 0.0988  0.0407
USUK       -0.0033 0.0864 -0.0385

#### 2. Implications for UIP:

(a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

Uncovered interest rate parity states that

$\log(\mathbb{E}_t[S_{t+1}]) - s_t = r^{f,\text{REF}}_{t+1} - r^{f,\text{QTE}}_{t+1}$

Our foreign exchange return series rearranges the equation to

$S_{t+1} - s_t - r^{f,\text{REF}}_{t+1} + r^{f,\text{QTE}}_{t+1} = 0$

Therefore, considering that there are non-zero means for the forex return series, we observe a contradiction of Uncovered Interest Rate parity. However, it can also be argued that the volatility is significantly higher than the mean, and the slight deviation from the expected value of 0 is a matter of noise.

(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?

CHF would have offered the best sharpe ratio by a significant margin.

(c) Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

EUR, JPY, and GBP all returned negative within the sample, likely due to the US economic boom in the 1990's.

#### 3. Predicting FX

For each foreign currency, test whether interest-rate differentials can predict growth in the
foreign-exchange rate.1 Do this by estimating the following forecasting regression::

where rf,i denotes the risk-free rate of currency i, and si denotes the FX rate for currency i.
Again, note that both rf,$ and st are determined at time t. t,t+1

(a) Make a table with columns corresponding to a different currency regression. Report the regression estimates αi and βi in the first two rows. Report the R2 stat in the third row.

In [11]:
dspot.dropna(inplace=True)
rates = rates.shift().dropna()
diff = -rates.subtract(rates["USD3M"], axis=0)

curr = [
    ("GBP3M", "USUK"),
    ("EUR3M", "USEU"), 
    ("CHF3M", "USSZ"),
    ("JPY3M", "USJP")
]
build_df = {
    "alpha": [],
    "beta": [],
    "rsq": []
}
resses = []

for t in curr:
    exo = sm.add_constant(diff[t[0]].loc[dspot.index])
    res = sm.OLS(dspot[t[1]], exo).fit()
    build_df["alpha"].append(res.params.values[0])
    build_df["beta"].append(res.params.values[1])
    build_df["rsq"].append(res.rsquared)
    resses.append(res)

rate_reg = pd.DataFrame(build_df, index=[c[1] for c in curr])
rate_reg["alpha"] *=12
display(rate_reg)

/Users/plee/miniconda3/envs/finmath/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta,rsq
USUK,-0.0061,0.4090,0.0003
USEU,0.0076,-1.3255,0.0031
USSZ,0.0462,-1.7743,0.0045
USJP,-0.0065,0.3845,0.0005


(b) Suppose the foreign risk-free rate increases relative to the US rate.

i. For which foreign currencies would we predict a relative strengthening of the USD in the following period?

The foreign risk free rate increasing decreases the exogenous variable. A relative strengthening of USD manifests in a negative value of the endogenous variable. Therefore, only JPY will yield such a result if the rate increase is large. However, GBP may also see relative strengthening if the rate increase is small enough to not overpower the regression intercept of -0.0007.

ii. For which currencies would we predict relative weakening of the USD in the following period?

EUR and CHF will certainly exhibit weakening of USD due to the regression yielding a positive return.

iii. This FX predictability is strongest in the case of which foreign currency?

The predictability is strongest in 1. CHF and 2. EUR due to relatively high $R^2$ statistics. GBP and JPY regressions hold very small predictive values by comparison.

#### 4. The Dynamic Carry Trade

Use this to write  as a function of the interest-rate differential as well as α and β from this FX regression.

Then use the definition of excess (log) returns on FX:

(a) Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. That is, for each i, calculate how often in the time-series we have

In [15]:
r_tilde = diff.iloc[:, 1:].multiply((rate_reg["beta"]-1).values, axis=1) \
                          .add(rate_reg["alpha"].values/12, axis=1)
display(r_tilde)

,GBP3M,EUR3M,CHF3M,JPY3M
DATE,,,,
1999-02-28,-0.0001,-0.0030,-0.0047,-0.0028
1999-03-31,-0.0003,-0.0031,-0.0048,-0.0030
1999-04-30,-0.0004,-0.0033,-0.0048,-0.0030
1999-05-31,-0.0003,-0.0040,-0.0054,-0.0030
1999-06-30,-0.0004,-0.0042,-0.0055,-0.0031
...,...,...,...,...
2021-06-30,-0.0005,-0.0007,0.0018,-0.0007
2021-07-31,-0.0005,-0.0007,0.0018,-0.0007
2021-08-31,-0.0005,-0.0007,0.0018,-0.0007


In [13]:
(r_tilde>0).sum() / len(r_tilde)

GBP3M   0.2454
EUR3M   0.5128
CHF3M   0.6190
JPY3M   0.0000
dtype: float64

(b) Which currencies most consistently have a positive FX risk premium? And for which
currencies does the FX risk premium most often go negative?

CHF has the most positive risk premium, JPY's risk premium is predicted to be 0 within all datapoints.

(c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

The predicted risk premia can be used to actively trade the carry, improving returns over passively holding.